In [1]:
import csv
import json
import pandas as pd
import numpy as np
import random
from collections import defaultdict

In [28]:
train_set = pd.read_csv('../data/final/train_reconstructed.csv')

X_train = train_set.iloc[:,:-1].values
y_train = train_set['edge'].values

print('Training set X: {}'.format(X_train[:2]))
print('Training set Y: {}'.format(y_train[:2]))

Training set X: [[ 7.         15.          7.          5.          2.          2.
   2.          0.          0.         12.          0.46153846  3.
   0.375     ]
 [ 7.          7.          7.          7.          2.          1.
   0.          1.          0.         13.          0.56521739  2.
   0.33333333]]
Training set Y: [1 1]


In [29]:
test_set = pd.read_csv('../data/final/dev-test.csv')

X_test = test_set.iloc[:,:-1].values
y_test = test_set['edge'].values

print('Test set X: {}'.format(X_test[:2]))
print('Test set Y: {}'.format(y_test[:2]))

Test set X: [[4.         5.         3.         3.         1.         0.
  0.         1.         0.         6.         0.3        0.
  0.        ]
 [8.         4.         2.         2.         0.         0.
  0.         0.         0.         8.         0.34782609 1.
  0.14285714]]
Test set Y: [0 1]


In [31]:
from sklearn.dummy import DummyClassifier

ds_clf = DummyClassifier(strategy="most_frequent") # Define our model, set parameter strategy to 'most_frequent'
ds_clf.fit(X_train, y_train) # Use model.fit to train with our dataset 
Y_predict = ds_clf.predict(X_test) # Use model.predict to make prediction
print("Prediction :", Y_predict[:10])
print("Accuracy:", ds_clf.score(X_test, y_test)) # Use model.score to evaluate our model.

Prediction : [1 1 1 1 1 1 1 1 1 1]
Accuracy: 0.5


In [41]:
from sklearn.naive_bayes import BernoulliNB

bnb = BernoulliNB()
bnb.fit(X_train, y_train)
Y_proba = bnb.predict_proba(X_test) # Use model.predict to make prediction
# Prob of being one
print("Probabilities :", Y_proba[:10,1])
print("Accuracy:", bnb.score(X_test, y_test)) # Use model.score to evaluate our model.


Probabilities : [0.00465919 0.87010758 0.78965432 0.78326649 0.00251905 0.00251905
 0.00251905 0.00251905 0.87010758 0.01462836]
Accuracy: 0.5281545417180435


0